In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService, Options, Sampler, Session, Estimator

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from torchquantum.measurement import expval_joint_analytical, expval_joint_sampling, expval_joint_sampling_grouping

In [2]:
import math
import pdb
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

import torchquantum as tq
import torchquantum.functional as tqf
from torchquantum.plugin.qiskit import tq2qiskit


In [5]:
from dotenv import dotenv_values

config = dotenv_values(".env")

In [6]:
service = QiskitRuntimeService(channel="ibm_quantum", token=config["IBM_TOKEN"])
options = Options(optimization_level=1, execution={"shots":32})

In [8]:
service.backends()

[<IBMBackend('simulator_statevector')>,
 <IBMBackend('simulator_stabilizer')>,
 <IBMBackend('ibm_kyoto')>,
 <IBMBackend('ibm_osaka')>,
 <IBMBackend('simulator_mps')>,
 <IBMBackend('simulator_extended_stabilizer')>,
 <IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_sherbrooke')>,
 <IBMBackend('ibmq_qasm_simulator')>]

In [9]:
backend = service.backend("ibmq_qasm_simulator")

----

Little test backend service and Estimator :

In [10]:
from qiskit import QuantumCircuit
 
qc = QuantumCircuit(2, 2)
qc.h(0)
#qc.h(1)
#qc.h(0)
#qc.h(1)
#qc.x(0)
qc.x(1)
#qc.cnot(0,1)
qc.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ X ├
     └───┘
c: 2/═════

In [11]:
qdev = tq.QuantumDevice(n_wires=3, bsz=1, device="cpu") # use device='cuda' for GPU


In [12]:
class TQTest(tq.QuantumModule):
        def __init__(self, n_qubits, *args, **kwargs):
            super().__init__()
            self.n_wires = n_qubits
            #self.rx_0 = tq.RX(has_params=True, trainable=True)
        @tq.static_support
        def forward(self, q_device, return_q_dev=False):
            tqf.h(q_device, wires=0, static=self.static_mode, parent_graph=self.graph)
            tqf.h(q_device, wires=2, static=self.static_mode, parent_graph=self.graph) 
            tqf.x(q_device, wires=1, static=self.static_mode, parent_graph=self.graph)
            #self.rx_0(q_device, wires=0)
             
            q_device = q_device.bfloat16()

            if return_q_dev:
                 return q_device


In [13]:
tqtest = TQTest(n_qubits=3)

In [14]:
for param in tqtest.parameters():
    print(param)

In [15]:
observables = ["III", "ZII","ZZI", "XXI", "IZI"]
sorted(expval_joint_sampling_grouping(qdev=tqtest(qdev, return_q_dev=True), observables=observables, n_shots_per_group=32).items())

[('III', tensor([1.])),
 ('IZI', tensor([-1.])),
 ('XXI', tensor([-0.1875])),
 ('ZII', tensor([-0.1875])),
 ('ZZI', tensor([0.1875]))]

In [16]:
qc_2 = tq2qiskit(q_device=qdev, m=tqtest)

In [17]:
qc.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ X ├
     └───┘
c: 2/═════

In [18]:
qc_2.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ X ├
     ├───┤
q_2: ┤ H ├
     └───┘

In [19]:
observables.sort()

In [20]:
observables

['III', 'IZI', 'XXI', 'ZII', 'ZZI']

In [21]:
observables = ["ZI", "IZ"]


with Session(service=service, backend=backend) as session:

    estimator = Estimator(session=session, options=options)
    job = estimator.run(circuits=[qc, qc], observables=observables)
    print(f"Estimator results: {job.result()}")

/tmp/ipykernel_6976/2304185545.py:7: DeprecationWarning: Circuits that do not match the target hardware definition will no longer be supported after March 1, 2024. See the transpilation documentation (https://docs.quantum.ibm.com/transpile) for instructions to transform circuits and the primitive examples (https://docs.quantum.ibm.com/run/primitives-examples) to see this coupled with operator transformations.
  job = estimator.run(circuits=[qc, qc], observables=observables)
/home/jdine/miniconda3/envs/retnet_test/lib/python3.12/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:935: UserWarning: Cloud simulators have been deprecated and will be removed on 15 May 2024. Use the new local testing mode in qiskit-ibm-runtime version 0.22.0 or later to meet your debugging needs.
  warnings.warn(warning_message)


Estimator results: EstimatorResult(values=array([-1.   ,  0.125]), metadata=[{'variance': 0.0, 'shots': 32}, {'variance': 0.984375, 'shots': 32}])


In [22]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
circ_test = pm.run(qc)

In [23]:
circ_test.draw()

global phase: π/4
     ┌─────────┐┌────┐┌─────────┐
q_0: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├
     └──┬───┬──┘└────┘└─────────┘
q_1: ───┤ X ├────────────────────
        └───┘                    
c: 2/════════════════════════════

In [24]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
circ_test_2 = pm.run(qc_2)

circ_test_2.draw()

global phase: π/2
     ┌─────────┐┌────┐┌─────────┐
q_0: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├
     └──┬───┬──┘└────┘└─────────┘
q_1: ───┤ X ├────────────────────
     ┌──┴───┴──┐┌────┐┌─────────┐
q_2: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├
     └─────────┘└────┘└─────────┘

In [11]:
observables = ["ZI", "IZ"]


with Session(service=service, backend=backend) as session:

    estimator = Estimator(session=session, options=options)
    job = estimator.run(circuits=[circ_test, circ_test], observables=observables)
    print(f"Estimator results: {job.result()}")

/tmp/ipykernel_11621/457931810.py:7: DeprecationWarning: Circuits that do not match the target hardware definition will no longer be supported after March 1, 2024. See the transpilation documentation (https://docs.quantum.ibm.com/transpile) for instructions to transform circuits and the primitive examples (https://docs.quantum.ibm.com/run/primitives-examples) to see this coupled with operator transformations.
  job = estimator.run(circuits=[circ_test, circ_test], observables=observables)
/home/jesshuan/miniconda3/envs/torch_quantum/lib/python3.12/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:935: UserWarning: Cloud simulators have been deprecated and will be removed on 15 May 2024. Use the new local testing mode in qiskit-ibm-runtime version 0.22.0 or later to meet your debugging needs.
  warnings.warn(warning_message)


Estimator results: EstimatorResult(values=array([-1.,  0.]), metadata=[{'variance': 0.0, 'shots': 32}, {'variance': 1.0, 'shots': 32}])


In [63]:
observables = ["ZI", "IZ"]


with Session(service=service, backend=backend) as session:

    estimator = Estimator(session=session, options=options)
    job = estimator.run(circuits=[circ_test_2, circ_test_2], observables=observables)
    print(f"Estimator results: {job.result()}")

/tmp/ipykernel_11621/1729889922.py:7: DeprecationWarning: Circuits that do not match the target hardware definition will no longer be supported after March 1, 2024. See the transpilation documentation (https://docs.quantum.ibm.com/transpile) for instructions to transform circuits and the primitive examples (https://docs.quantum.ibm.com/run/primitives-examples) to see this coupled with operator transformations.
  job = estimator.run(circuits=[circ_test_2, circ_test_2], observables=observables)
/home/jesshuan/miniconda3/envs/torch_quantum/lib/python3.12/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:935: UserWarning: Cloud simulators have been deprecated and will be removed on 15 May 2024. Use the new local testing mode in qiskit-ibm-runtime version 0.22.0 or later to meet your debugging needs.
  warnings.warn(warning_message)


Estimator results: EstimatorResult(values=array([-1.,  0.]), metadata=[{'variance': 0.0, 'shots': 32}, {'variance': 1.0, 'shots': 32}])


In [87]:
job.result().values

array([-1.   ,  0.125])